<a href="https://colab.research.google.com/github/m-wessler/nbm-verification/blob/main/NBM_PoWT_Verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Imports etc
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# INSTALL AND IMPORTS                                                         #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

!pip install boto3
!pip install pygrib
!pip install swifter

import gc
import os
import time
import json
import boto3
import pygrib
import swifter
import zipfile
import requests
import itertools

import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from glob import glob
from typing import List, Optional
from pydantic import BaseModel, Field

from functools import partial
from google.colab import files
from matplotlib.gridspec import GridSpec
from datetime import datetime, timedelta
from collections import defaultdict, OrderedDict

from multiprocessing import Pool, cpu_count
from multiprocessing import set_start_method, get_context

from sklearn.metrics import RocCurveDisplay
from sklearn.calibration import CalibrationDisplay
from sklearn.metrics import (brier_score_loss, f1_score, log_loss,
                                precision_score, recall_score, roc_auc_score)

import warnings
warnings.filterwarnings('ignore')

idx = pd.IndexSlice

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 18.7 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16506 sha256=2fb7b2d52252c6ea41ce815b58dec6035397fe373dfef3f3809a5393d527724e
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [3]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# COLAB MARKDOWN AND USER CONFIGS                                             #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

# @markdown <FONT SIZE=5>**1. Please Provide Your Synoptic API Token...**
user_token = "a2386b75ecbc4c2784db1270695dde73" # @param {type:"string"}

# @markdown <FONT SIZE=5>**2. Select Start and End Dates**
start_date = "2023-12-15" # @param {type:"date"}
end_date = "2024-01-15" # @param {type:"date"}

## @markdown <FONT SIZE=5>**3. For Which Element?**
##"qpf24" # @param ["maxt", "mint", "qpf24", "qpf12", "qpf06"]
element = "powt"

# Split element/interval
# interval_selection = int(element[-2:]) if "qpf" in element else False
# element = element[:3] if "qpf" in element else element

#6/12/24/48/72, if element==temp then False
# interval_selection = "24" #@param ["24", "12", "6"]
# interval_selection = interval_selection if element == "qpf" else False

#temperature_threshold = -60 #@param {type:"slider", min:-60, max:140, step:10}
#qpf_threshold = 0.31 #@param {type:"slider", min:0.01, max:5.00, step:0.01}

#if element in ["maxt","mint"]:
#    threshold = temperature_threshold
#elif element in ["qpf"]:
#    threshold = qpf_threshold

# @markdown <FONT SIZE=5>**3. Which NBM Text Product?**
nbm_text_set = 'nbs' #@param ["nbh", "nbs", "nbe", "nbx"]
tolerance = 90 #@param {type:"slider", min:0, max:90, step:15}

# @markdown <FONT SIZE=5>**4. For Which Lead Time (in days)?**
lead_days_selection = 1 #@param {type:"slider", min:1, max:8, step:1}

# @markdown <FONT SIZE=5>**5. For Which Region?**
region_selection = "CWA" #@param ["WR", "SR", "CR", "ER", "CONUS", "CWA", "RFC"]

#@markdown If CWA/RFC selected, which one? (i.e. "SLC" for Salt Lake City, "CBRFC" for Colorado Basin)
cwa_selection = 'SEW,PQR,PDT,OTX,MFR' #@param {type:"string"}

## @markdown For Which Networks?
#network_selection = 'NWS+RAWS+HADS'#@param ["NWS+RAWS", "NWS+RAWS+HADS", "NWS", "RAWS", "HADS", "SNOTEL", "ALL"]
network_selection = 'NWS'

# @markdown Check box to display plots inline (default unchecked for download)
display_inline = False #@param {type:"boolean"}

In [4]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# GLOBAL VARIABLES AND GENERAL CONFIG                                         #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

# Multiprocess settings
process_pool_size = 20 #cpu_count()*16
print(f'Process Pool Size: {process_pool_size}')

# Backend APIs
metadata_api = "https://api.synopticdata.com/v2/stations/metadata?"
qc_api = "https://api.synopticdata.com/v2/stations/qcsegments?"

# Data Query APIs
timeseries_api = "https://api.synopticdata.com/v2/stations/timeseries?"
statistics_api = "https://api.synopticlabs.org/v2/stations/statistics?"
precipitation_api = "https://api.synopticdata.com/v2/stations/precipitation?"

# Assign API to element name
synoptic_apis = {
    'qpf':precipitation_api,
    'maxt':statistics_api,
    'mint':statistics_api,
    'powt':timeseries_api}

synoptic_networks = {"NWS+RAWS+HADS":"1,2,106",
                     "NWS+RAWS":"1,2",
                     "NWS":"1",
                     "RAWS": "2",
                     "HADS": "106",
                     "SNOTEL":"25",
                     "ALL":None}
                    #  "CUSTOM": "&network="+network_input,
                    #  "LIST": "&stid="+network_input}

# Assign synoptic variable to element name
synoptic_vars = {'powt':'weather_condition,weather_summary,' \
        'weather_cond_code_synop,past_weather_code,weather_cond_code,air_temp'}

synoptic_vars_out = {
    'powt':''}

# Assign stat type to element name
stat_type = {
    'powt':None}

ob_hours = {
    'powt':['0000']}

# Where to place the grib files (subdirs can be added in local) (not used)
output_dir = './' #/nas/stid/data/nbm-verification/temp/'

# Convert user input to datetime objects
start_date, end_date = [datetime.strptime(date+' 0000', '%Y-%m-%d %H%M')
    for date in [start_date, end_date]]

# Build synoptic arg dict
synoptic_api_args = {
    'ob_stat':stat_type[element],
    'api':synoptic_apis[element],
    'element':element,
    'interval':24, #24h to poll daily powt data
    #interval_selection if element == 'qpf' else False,
    'region':region_selection,
    'network_query':synoptic_networks[network_selection], # add config feature later
    'vars_query':None if element == 'qpf'
        else f'{synoptic_vars[element]}',}

Process Pool Size: 20


In [5]:
class PeriodOfRecord(BaseModel):
    start: str
    end: str

class Providers(BaseModel):
    name: str
    url: str

class Units(BaseModel):
    position: str
    elevation: str

class SensorVariables(BaseModel):
    air_temp: dict = Field(default_factory=dict)
    weather_summary: dict = Field(default_factory=dict)
    weather_condition: dict = Field(default_factory=dict)
    weather_cond_code: dict = Field(default_factory=dict)
    past_weather_code: dict = Field(default_factory=dict)
    weather_cond_code_synop: dict = Field(default_factory=dict)

class Observations(BaseModel):
    date_time: List[str]

    air_temp_set_1: List[Optional[float]] = Field(default_factory=list)
    air_temp_set_1d: List[Optional[float]] = Field(default_factory=list)

    weather_summary_set_1: List[Optional[str]] = Field(default_factory=list)
    weather_summary_set_1d: List[Optional[str]] = Field(default_factory=list)

    weather_condition_set_1: List[Optional[str]] = Field(default_factory=list)
    weather_condition_set_1d: List[Optional[str]] = Field(default_factory=list)

    weather_cond_code_set_1: List[Optional[float]] = Field(default_factory=list)
    weather_cond_code_set_1d: List[Optional[float]] = Field(default_factory=list)

    weather_cond_code_synop_set_1: List[Optional[object]] = Field(default_factory=list)
    weather_cond_code_synop_set_1d: List[Optional[object]] = Field(default_factory=list)

    past_weather_code_set_1: List[Optional[object]] = Field(default_factory=list)
    past_weather_code_set_1d: List[Optional[object]] = Field(default_factory=list)

    class Config:
        allow_population_by_field_name = True
        arbitrary_types_allowed = True
        strict = False

class Station(BaseModel):
    ID: str
    STID: str
    NAME: str
    ELEVATION: float
    LATITUDE: float
    LONGITUDE: float
    STATUS: str
    MNET_ID: str
    STATE: str
    TIMEZONE: str
    ELEV_DEM: object
    # NWSZONE: str
    # NWSFIREZONE: str
    # GACC: str
    SHORTNAME: str
    # SGID: str
    COUNTY: str
    # COUNTRY: str
    # WIMS_ID: str
    CWA: str
    PERIOD_OF_RECORD: PeriodOfRecord
    # PROVIDERS: Providers
    UNITS: Units
    SENSOR_VARIABLES: SensorVariables
    OBSERVATIONS: Observations
    QC_FLAGGED: bool
    RESTRICTED: bool

class Data(BaseModel):
    STATION: List[Station]

    def to_dataframe(self) -> pd.DataFrame:
        records = []

        for station in self.STATION:
            for i, date_time in enumerate(station.OBSERVATIONS.date_time):
                record = {
                    'stid': station.STID,
                    'latitude': station.LATITUDE,
                    'longitude': station.LONGITUDE,
                    'elevation': station.ELEVATION,
                    'state': station.STATE,
                    'cwa': station.CWA,
                    'timestamp':date_time,

                    'air_temp_set_1': station.OBSERVATIONS.air_temp_set_1[i] if len(station.OBSERVATIONS.air_temp_set_1) > i else None,
                    'weather_summary_set_1': station.OBSERVATIONS.weather_summary_set_1[i] if len(station.OBSERVATIONS.weather_summary_set_1) > i else None,
                    'weather_condition_set_1': station.OBSERVATIONS.weather_condition_set_1[i] if len(station.OBSERVATIONS.weather_condition_set_1) > i else None,
                    'weather_cond_code_set_1': station.OBSERVATIONS.weather_cond_code_set_1[i] if len(station.OBSERVATIONS.weather_cond_code_set_1) > i else None,
                    'past_weather_code_set_1': station.OBSERVATIONS.past_weather_code_set_1[i] if len(station.OBSERVATIONS.past_weather_code_set_1) > i else None,
                    'weather_cond_code_synop_set_1': station.OBSERVATIONS.weather_cond_code_synop_set_1[i] if len(station.OBSERVATIONS.weather_cond_code_synop_set_1) > i else None,

                    'air_temp_set_1d': station.OBSERVATIONS.air_temp_set_1d[i] if len(station.OBSERVATIONS.air_temp_set_1d) > i else None,
                    'weather_summary_set_1d': station.OBSERVATIONS.weather_summary_set_1d[i] if len(station.OBSERVATIONS.weather_summary_set_1d) > i else None,
                    'weather_condition_set_1d': station.OBSERVATIONS.weather_condition_set_1d[i] if len(station.OBSERVATIONS.weather_condition_set_1d) > i else None,
                    'weather_cond_code_set_1d': station.OBSERVATIONS.weather_cond_code_set_1d[i] if len(station.OBSERVATIONS.weather_cond_code_set_1d) > i else None,
                    'past_weather_code_set_1d': station.OBSERVATIONS.past_weather_code_set_1d[i] if len(station.OBSERVATIONS.past_weather_code_set_1d) > i else None,
                    'weather_cond_code_synop_set_1d': station.OBSERVATIONS.weather_cond_code_synop_set_1d[i] if len(station.OBSERVATIONS.weather_cond_code_synop_set_1d) > i else None
                }
                records.append(record)

        df = pd.DataFrame(records)
        return df

In [6]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# FUNCTIONS AND METHODS (GENERAL)                                             #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

def mkdir_p(check_dir):
    from pathlib import Path
    check_dir = output_dir + check_dir
    Path(check_dir).mkdir(parents=True, exist_ok=True)
    return check_dir

In [7]:
def cwa_list(input_region):

    input_region = input_region.upper()

    region_dict ={
        "WR":["BYZ", "BOI", "LKN", "EKA", "FGZ", "GGW", "TFX", "VEF", "LOX", "MFR",
            "MSO", "PDT", "PSR", "PIH", "PQR", "REV", "STO", "SLC", "SGX", "MTR",
            "HNX", "SEW", "OTX", "TWC"],

        "CR":["ABR", "BIS", "CYS", "LOT", "DVN", "BOU", "DMX", "DTX", "DDC", "DLH",
            "FGF", "GLD", "GJT", "GRR", "GRB", "GID", "IND", "JKL", "EAX", "ARX",
            "ILX", "LMK", "MQT", "MKX", "MPX", "LBF", "APX", "IWX", "OAX", "PAH",
            "PUB", "UNR", "RIW", "FSD", "SGF", "LSX", "TOP", "ICT"],

        "ER":["ALY", "LWX", "BGM", "BOX", "BUF", "BTV", "CAR", "CTP", "RLX", "CHS",
            "ILN", "CLE", "CAE", "GSP", "MHX", "OKX", "PHI", "PBZ", "GYX", "RAH",
            "RNK", "AKQ", "ILM"],

        "SR":["ABQ", "AMA", "FFC", "EWX", "BMX", "BRO", "CRP", "EPZ", "FWD", "HGX",
            "HUN", "JAN", "JAX", "KEY", "MRX", "LCH", "LZK", "LUB", "MLB", "MEG",
            "MAF", "MFL", "MOB", "MRX", "OHX", "LIX", "OUN", "SJT", "SHV", "TAE",
            "TBW", "TSA"]}

    if input_region == "CONUS":
        return np.hstack([region_dict[region] for region in region_dict.keys()])
    else:
        return region_dict[input_region]

def cwa_list_rfc(input_rfc):

    metadata_api = 'https://api.synopticdata.com/v2/stations/metadata?'

    network_query = (f"&network={synoptic_networks[network_selection]}"
                    if synoptic_networks[network_selection] is not None else '')

    # Assemble the API query
    api_query = (f"{metadata_api}&token={user_token}" + network_query +
                f"&complete=1&sensorvars=1,obrange=20230118") #hardcoded for NBM4.1+

    # Print the API query to output
    # print(api_query)

    # Get the data from the API
    response = requests.get(api_query)
    metadata = pd.DataFrame(response.json()['STATION'])

    # Remove NaNs and index by network, station ID
    metadata = metadata[metadata['MNET_SHORTNAME'].notna()]
    metadata = metadata.set_index(['MNET_SHORTNAME', 'STID'])

    metadata['LATITUDE'] = metadata['LATITUDE'].astype(float)
    metadata['LONGITUDE'] = metadata['LONGITUDE'].astype(float)
    metadata['ELEVATION'] = metadata['ELEVATION'].astype(float)

    metadata = metadata[metadata['LATITUDE'] >= 31]
    metadata = metadata[metadata['LONGITUDE'] <= -103.00]
    metadata = metadata[metadata['STATUS'] == 'ACTIVE']

    geometry = gpd.points_from_xy(metadata.LONGITUDE, metadata.LATITUDE)
    metadata = gpd.GeoDataFrame(metadata, geometry=geometry)

    req = requests.get(
        'https://www.weather.gov/source/gis/Shapefiles/Misc/rf05mr24.zip',

    allow_redirects=True)
    open('rf05mr24.zip', 'wb').write(req.content)

    with zipfile.ZipFile('rf05mr24.zip', 'r') as zip_ref:
        zip_ref.extractall()

    rfc_shp = gpd.read_file('rf05mr24.shp').set_index('BASIN_ID')

    metadata = metadata[metadata.geometry.within(rfc_shp.geometry.loc[input_rfc])]

    rfc_site_list = metadata.index.get_level_values(1).unique()
    rfc_cwa_list = metadata['CWA'].unique()

    return metadata

In [8]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# FUNCTIONS AND METHODS (SYNOPTIC API)                                        #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

def fetch_obs_from_API(valid_datetime, cwa='', output_type='json',
                       use_saved=True, **req):

    start_adjusted = (datetime.strptime(valid_datetime, '%Y%m%d%H%M')
                        - timedelta(hours=req["interval"]))

    end_adjusted = datetime.strptime(valid_datetime, '%Y%m%d%H%M')

    valid = True
    cwa_filename = (region_selection if region_selection != 'CWA'
                    else cwa_selection)

    element_label = req['element'] if req['element'] != 'qpf' else \
                        'qpe' + f'{req["interval"]:02d}'


    output_file = mkdir_p(f'obs_{output_type}/') +\
        f'obs.{element_label}.{req["ob_stat"]}' +\
        f'.{valid_datetime}.{cwa_filename}.{output_type}'

    if os.path.isfile(output_file) & use_saved:
        print(f'Output file exists for:{iter_item}')

        with open(output_file, 'r') as f:
            data = Data.model_validate_json(f.read())

            # Convert to DataFrame
            df = data.to_dataframe()
            df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize(None)

            return df

    else:
        json_file = mkdir_p('obs_json/') +\
            f'obs.{element_label}.{req["ob_stat"]}' +\
            f'.{valid_datetime}.{cwa_filename}.json'

        # if os.path.isfile(json_file) & use_saved:
        #     # print(f'Polling archived JSON for: {iter_item}')

        #     with open(json_file, 'rb+') as rfp:
        #         response_dataframe = pd.json_normalize(json.load(rfp)['STATION'])

        # else:
        api_query_args = {
            'api_token':f'&token={user_token}',
            'station_query':f'&cwa={cwa}',
            'network_query':(f'&network={req["network_query"]}'
                                if req["network_query"] is not None else ''),

            'start_date_query':f'&start={start_adjusted.strftime("%Y%m%d%H%M")}',
            'end_date_query':f'&end={end_adjusted.strftime("%Y%m%d%H%M")}',

            'vars_query':(f'&pmode=intervals&interval={req["interval"]}'
                            if req["element"] == 'qpf'
                            else f'&vars={req["vars_query"]}'),
            'stats_query':f'&type={req["ob_stat"]}',
            'timezone_query':'&obtimezone=utc',
            'api_extras':'&units=temp|f&complete=True'}
                #'&fields=name,status,latitude,longitude,elevation'

        api_query = req['api'] + ''.join(
            [api_query_args[k] for k in api_query_args.keys()])

        print(f'Polling API for: {iter_item}\n{api_query}')

        status_code, response_count = None, 0
        while (status_code != 200) & (response_count <= 10):
            print(f'{iter_item}, HTTP:{status_code}, #:{response_count}')

            # Don't sleep first try, sleep increasing amount for each retry
            time.sleep(2*response_count)

            response = requests.get(api_query)
            # response.raise_for_status()

            status_code = response.status_code
            response_count += 1

        try:
            response_dataframe = pd.json_normalize(
                response.json()['STATION'])
        except:
            valid = False
        else:
            with open(json_file, 'wb+') as wfp:
                wfp.write(response.content)

    if valid:
        # Example usage
        # with open('sample.json', 'r') as f:
        data = Data.model_validate_json(response.content)

        # Convert to DataFrame
        df = data.to_dataframe()
        df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize(None)

        return df

    else:
        return None

In [9]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# INPUT-BASED GLOBAL VARIABLES AND CONFIG                                     #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

# Build an iterable date list from range
iter_date = start_date
valid_date_iterable = []
valid_datetime_iterable = []
forecast_datetime_iterable = []

while iter_date <= end_date:

    valid_date_iterable.append(iter_date.strftime('%Y%m%d'))

    for hour_range in ob_hours[element]:
        end_hour = f'{int(hour_range[-1]):02d}'

        valid_datetime_iterable.append(iter_date.strftime('%Y%m%d') + end_hour)

        forecast_datetime_iterable.append(
                (iter_date-timedelta(days=lead_days_selection)
            ).strftime('%Y%m%d') + end_hour)

    iter_date += timedelta(days=1)

# Assign the fixed kwargs to the function
if region_selection == 'CWA':
    cwa_query = cwa_selection
elif region_selection == 'RFC':
    rfc_metadata = cwa_list_rfc(cwa_selection)
    cwa_query = ','.join([str(cwa) for cwa in rfc_metadata['CWA'].unique()
                if cwa is not None])
else:
    cwa_query = ','.join(cwa_list(region_selection))

In [10]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# DATA ACQUISITION                                                            #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
multiprocess_function = partial(fetch_obs_from_API,
                                cwa=cwa_query,
                                **synoptic_api_args)

df_list = []
# Multithreaded requests currently not supported by the Synoptic API
for iter_item in valid_datetime_iterable:
    df_list.append(multiprocess_function(iter_item))

df = pd.concat(df_list).set_index(['timestamp', 'stid']).sort_index().drop_duplicates()

# Combine similarly named columns
for col in df.columns:
    if col.endswith('_1'):
        new_col_name = col[:-6]  # Remove the last two characters (_1)
        df[new_col_name] = df[col].combine_first(df[col + 'd'])  # Keep non-NaN values

# Drop the original _1 and _1d columns if needed
obs = df.drop(columns=[col for col in df.columns if col.endswith(('_1', '_1d'))])

# Display the resulting DataFrame
obs

Polling API for: 2023121500
https://api.synopticdata.com/v2/stations/timeseries?&token=a2386b75ecbc4c2784db1270695dde73&cwa=SEW,PQR,PDT,OTX,MFR&network=1&start=202312140000&end=202312150000&vars=weather_condition,weather_summary,weather_cond_code_synop,past_weather_code,weather_cond_code,air_temp&type=None&obtimezone=utc&units=temp|f&complete=True
2023121500, HTTP:None, #:0
Polling API for: 2023121600
https://api.synopticdata.com/v2/stations/timeseries?&token=a2386b75ecbc4c2784db1270695dde73&cwa=SEW,PQR,PDT,OTX,MFR&network=1&start=202312150000&end=202312160000&vars=weather_condition,weather_summary,weather_cond_code_synop,past_weather_code,weather_cond_code,air_temp&type=None&obtimezone=utc&units=temp|f&complete=True
2023121600, HTTP:None, #:0
Polling API for: 2023121700
https://api.synopticdata.com/v2/stations/timeseries?&token=a2386b75ecbc4c2784db1270695dde73&cwa=SEW,PQR,PDT,OTX,MFR&network=1&start=202312160000&end=202312170000&vars=weather_condition,weather_summary,weather_cond_code

latitude  longitude  elevation state  cwa air_temp  \
timestamp           stid                                                       
2023-12-14 00:00:00 KAAT  41.48362 -120.56150     4383.0    CA  MFR     42.8   
                    KALW  46.09462 -118.28580     1168.0    WA  PDT     39.2   
                    KAWO  48.15850 -122.15909      125.0    WA  SEW     46.4   
                    KBFI  47.54548 -122.31470       20.0    WA  SEW     48.2   
                    KBLI  48.79911 -122.54064      148.0    WA  SEW     41.0   
...                            ...        ...        ...   ...  ...      ...   
2024-01-14 23:55:00 KHRI  45.82583 -119.26111      636.0    OR  PDT     10.4   
                    KSKA  47.61667 -117.65000     2461.0    WA  OTX     7.16   
2024-01-14 23:56:00 KMEH  45.51176 -118.42466     3730.0    OR  PDT     8.06   
2024-01-15 00:00:00 KCLM  48.12194 -123.50661      262.0    WA  SEW     32.0   
                    KOMK  48.46116 -119.52086     1293.0    WA  OTX      8.6   

                         weather_summary weather_condition weather_cond_code  \
timestamp           stid                                                       
2023-12-14 00:00:00 KAAT           clear              None              None   
                    KALW           clear              None              None   
                    KAWO  thin scattered              None              None   
                    KBFI           clear              None              None   
                    KBLI        overcast              None              None   
...                                  ...               ...               ...   
2024-01-14 23:55:00 KHRI          broken              None              None   
                    KSKA           clear              None              None   
2024-01-14 23:56:00 KMEH        overcast              None              None   
2024-01-15 00:00:00 KCLM        overcast              None              None   
                    KOMK           clear              None              None   

                         past_weather_code weather_cond_code_synop  
timestamp           stid                                            
2023-12-14 00:00:00 KAAT              None                    None  
                    KALW              None                    None  
                    KAWO              None                    None  
                    KBFI              None                    None  
                    KBLI              None                    None  
...                                    ...                     ...  
2024-01-14 23:55:00 KHRI              None                    None  
                    KSKA              None                    None  
2024-01-14 23:56:00 KMEH              None                    None  
2024-01-15 00:00:00 KCLM              None                    None  
                    KOMK              None                    None  

[28074 rows x 11 columns]

In [11]:
# Read in the model data to verify against - downsample to the model timestep
nbm = pd.concat([pd.read_csv(f) for f in glob('./csv/*.csv')])
nbm.rename(columns={k:k.replace(' ','') for k in nbm.columns}, inplace=True)
nbm.rename(columns={'datetime':'nbm_timestamp', 'site_id':'stid'}, inplace=True)
nbm['nbm_timestamp'] = pd.to_datetime(nbm['nbm_timestamp']).dt.tz_localize(None)

nbm = nbm[['nbm_timestamp', 'stid', 'TMP', 'PZR', 'PSN', 'PPL', 'PRA']]
nbm.set_index(['nbm_timestamp', 'stid'], inplace=True)

stid_slice = obs.index.get_level_values('stid').unique()
stid_slice = np.intersect1d(stid_slice, nbm.index.get_level_values(1))

nbm = nbm.loc[idx[:, stid_slice], :].sort_index()
nbm

TMP  PZR   PSN  PPL   PRA
nbm_timestamp       stid                            
2023-12-14 18:00:00 K0S9  45.0  0.0   0.0  0.0  12.0
                    K1S5  37.0  0.0   0.0  0.0  12.0
                    K3S8  42.0  0.0   0.0  0.0   0.0
                    K4S1  52.0  0.0   0.0  0.0   2.0
                    K4S2  39.0  0.0   0.0  0.0  12.0
...                        ...  ...   ...  ...   ...
2024-01-19 00:00:00 KTTD  40.0  6.0   0.0  1.0  78.0
                    KUAO  43.0  2.0   0.0  0.0  82.0
                    KUIL  45.0  0.0   1.0  0.0  95.0
                    KVUO  39.0  6.0   0.0  1.0  78.0
                    KYKM  28.0  0.0  69.0  0.0   6.0

[58650 rows x 5 columns]

In [13]:
obs_ts = obs.index.get_level_values(0)
nbm_ts = nbm.index.get_level_values(0).unique()

for ti in nbm_ts:
    mask = np.where(abs(obs_ts-ti) < timedelta(minutes=tolerance), True, False)
    obs.loc[mask, 'nbm_timestamp'] = ti

obs.dropna(subset=['nbm_timestamp'], inplace=True)
obs = obs.reset_index().set_index(['nbm_timestamp', 'stid']).sort_index()

# Broadcast the NBM data ONTO the observation dataframe!
obs = obs.join(nbm).reset_index().set_index(['nbm_timestamp', 'stid', 'timestamp']).sort_index()
obs

latitude  longitude  elevation  \
nbm_timestamp       stid timestamp                                             
2023-12-14 18:00:00 K1S5 2023-12-14 17:55:00  46.32710 -119.97040      768.0   
                         2023-12-14 18:15:00  46.32710 -119.97040      768.0   
                         2023-12-14 18:35:00  46.32710 -119.97040      768.0   
                         2023-12-14 19:15:00  46.32710 -119.97040      768.0   
                    K4S1 2023-12-14 16:35:00  42.41536 -124.42492       16.0   
...                                                ...        ...        ...   
2024-01-15 00:00:00 KTCM 2024-01-14 22:55:00  47.15000 -122.48333      322.0   
                         2024-01-14 22:55:00  47.15000 -122.48333      322.0   
                    KUAO 2024-01-14 23:10:00  45.24658 -122.77095      194.0   
                         2024-01-14 23:10:00  45.24658 -122.77095      194.0   
                         2024-01-14 23:10:00  45.24658 -122.77095      194.0   

                                             state  cwa air_temp  \
nbm_timestamp       stid timestamp                                 
2023-12-14 18:00:00 K1S5 2023-12-14 17:55:00    WA  PDT     37.4   
                         2023-12-14 18:15:00    WA  PDT    37.76   
                         2023-12-14 18:35:00    WA  PDT     38.3   
                         2023-12-14 19:15:00    WA  PDT    39.56   
                    K4S1 2023-12-14 16:35:00    OR  MFR     46.4   
...                                            ...  ...      ...   
2024-01-15 00:00:00 KTCM 2024-01-14 22:55:00    WA  SEW    31.28   
                         2024-01-14 22:55:00    WA  SEW    31.28   
                    KUAO 2024-01-14 23:10:00    OR  PQR     24.8   
                         2024-01-14 23:10:00    OR  PQR     24.8   
                         2024-01-14 23:10:00    OR  PQR     24.8   

                                             weather_summary  \
nbm_timestamp       stid timestamp                             
2023-12-14 18:00:00 K1S5 2023-12-14 17:55:00        overcast   
                         2023-12-14 18:15:00          broken   
                         2023-12-14 18:35:00          broken   
                         2023-12-14 19:15:00        overcast   
                    K4S1 2023-12-14 16:35:00            mist   
...                                                      ...   
2024-01-15 00:00:00 KTCM 2024-01-14 22:55:00           clear   
                         2024-01-14 22:55:00           clear   
                    KUAO 2024-01-14 23:10:00          broken   
                         2024-01-14 23:10:00          broken   
                         2024-01-14 23:10:00          broken   

                                             weather_condition  \
nbm_timestamp       stid timestamp                               
2023-12-14 18:00:00 K1S5 2023-12-14 17:55:00              None   
                         2023-12-14 18:15:00              None   
                         2023-12-14 18:35:00              None   
                         2023-12-14 19:15:00              None   
                    K4S1 2023-12-14 16:35:00              mist   
...                                                        ...   
2024-01-15 00:00:00 KTCM 2024-01-14 22:55:00              None   
                         2024-01-14 22:55:00              None   
                    KUAO 2024-01-14 23:10:00              None   
                         2024-01-14 23:10:00              None   
                         2024-01-14 23:10:00              None   

                                             weather_cond_code  \
nbm_timestamp       stid timestamp                               
2023-12-14 18:00:00 K1S5 2023-12-14 17:55:00              None   
                         2023-12-14 18:15:00              None   
                         2023-12-14 18:35:00              None   
                         2023-12-14 19:15:00              None   
                    K4S1 2023-12